In [1]:
import pandas as pd
from hashlib import sha512

pathprefix = r"D:\Code Working Area\Jupyter\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData"
jsfile = pathprefix+"\\Transactions.json"
stop_words=pathprefix+"\\stop_words\\stop_words_stakeholder.txt"
unknow_long_phrase=pathprefix+"\\stop_words\\unknow_long.txt"

project_sor = pathprefix+"\\Source\\project.csv"
risk_sor_path = pathprefix+"\\Source\\risk.xlsx"
stake_sor = pathprefix+"\\Source\\stakeholder.csv"

project_key = pathprefix+"\\project_keyword\\Project_keyword.xlsx"
risk_key_path = pathprefix+"\\risk_keyword\\Risk_keyword.xlsx"
stake_key = pathprefix+"\\stakeholder_keyword\\third_layer_iteration_one_stakeholder.xlsx"

In [2]:
prj_sor = pd.read_csv(project_sor, sep = ",")
risk_sor = pd.read_excel(risk_sor_path)
stk_sor = pd.read_csv(stake_sor, sep = ",")

def sample_filter(df: pd.DataFrame, columns: str = "Abstract", n: int = 10_000) -> pd.DataFrame:
    return df[columns].to_frame().dropna(how="any").reset_index(drop=True).sample(n=n, random_state=42) # .loc[:30_000, "Abstract"].to_frame()

C:\Users\jqi22\AppData\Local\Temp\ipykernel_13460\3794204323.py:1: DtypeWarning: Columns (3,7,8,12,13,18,27,28,29,30,31,38,39,40,41,42,43,44,45,50,51,53,54,59,63,64,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  prj_sor = pd.read_csv(project_sor, sep = ",")


In [3]:
with open(r"input/mix_30K.txt", "r") as file:
    test_set = [lines.rstrip("\n") for lines in file.readlines() if lines!="\n"]

In [4]:
# test_set = set(test_set)
# prj, stk, risk = set(prj_sor["Abstract"].to_list()), set(stk_sor["Abstract"].to_list()), set(risk_sor["Abstract"].to_list())
# prj_left = risk.difference(test_set)

In [5]:
prj_sor["Abstract"].shape

(266327,)

In [6]:
prj, stk, risk = prj_sor.loc[:, "Abstract"].dropna().to_frame(), stk_sor.loc[:, "Abstract"].dropna().to_frame(), risk_sor.loc[:, "Abstract"].dropna().to_frame()
test_set = pd.DataFrame({"test": test_set})
oprj, ostk, orisk, otest = prj.shape, stk.shape, risk.shape, test_set.shape

In [7]:
prj, stk, risk = prj.Abstract.apply(lambda x: sha512(x.encode()).hexdigest()).drop_duplicates().to_frame(), stk.Abstract.apply(lambda x: sha512(x.encode()).hexdigest()).drop_duplicates().to_frame(), risk.Abstract.apply(lambda x: sha512(x.encode()).hexdigest()).drop_duplicates().to_frame()
test_set = test_set.test.apply(lambda x: sha512(x.encode()).hexdigest()).to_frame()

In [8]:
prj_ndup, stk_ndup, risk_ndup = prj[~prj["Abstract"].isin(test_set["test"])], \
    stk[~stk["Abstract"].isin(test_set["test"])], \
        risk[~risk["Abstract"].isin(test_set["test"])]

In [9]:
print(f"the original shape of project compare with filtered document is {oprj[0], prj_ndup.shape[0], round(prj_ndup.shape[0]/oprj[0], 2)} \n",\
      f"the original shape of stakeholder compare with filtered document is {ostk[0], stk_ndup.shape[0], round(stk_ndup.shape[0]/ostk[0], 2)} \n",\
    f"the original shape of risk compare with filtered document is {orisk[0], risk_ndup.shape[0], round(risk_ndup.shape[0]/orisk[0], 2)} \n")

the original shape of project compare with filtered document is (254352, 135053, 0.53) 
 the original shape of stakeholder compare with filtered document is (48341, 45075, 0.93) 
 the original shape of risk compare with filtered document is (120420, 106322, 0.88) 



In [10]:
filtered_prj = prj[~(prj['Abstract'].isin(stk['Abstract']) | prj['Abstract'].isin(risk['Abstract']))]

filtered_stk = stk[~(stk['Abstract'].isin(prj['Abstract']) | stk['Abstract'].isin(risk['Abstract']))]

filtered_risk = risk[~(risk['Abstract'].isin(prj['Abstract']) | risk['Abstract'].isin(stk['Abstract']))]

In [11]:
real_prj, real_stk, real_risk = prj_sor.loc[filtered_prj.index, "Abstract"], stk_sor.loc[filtered_stk.index, "Abstract"], risk_sor.loc[filtered_risk.index, "Abstract"]

In [93]:
real_prj.to_csv(r"input/non_duplicate_prj.csv", index=False)
real_stk.to_csv(r"input/non_duplicate_stk.csv", index=False)
real_risk.to_csv(r"input/non_duplicate_risk.csv", index=False)

In [12]:
print(f"the original shape of project compare with filtered document is {oprj[0], real_prj.shape[0], round(real_prj.shape[0]/oprj[0], 2)} \n",\
      f"the original shape of stakeholder compare with filtered document is {ostk[0], real_stk.shape[0], round(real_stk.shape[0]/ostk[0], 2)} \n",\
    f"the original shape of risk compare with filtered document is {orisk[0], real_risk.shape[0], round(real_risk.shape[0]/orisk[0], 2)} \n")

the original shape of project compare with filtered document is (254352, 146048, 0.57) 
 the original shape of stakeholder compare with filtered document is (48341, 35012, 0.72) 
 the original shape of risk compare with filtered document is (120420, 94860, 0.79) 



In [100]:
with open(r"input/risk.txt", "a+") as file:
    for val in real_risk.to_list(): file.writelines(val+"\n")

In [98]:
real_prj.tolist()[:5]

["This paper considers how an offset scheme specific to the electricity generation sector could incentivize emissions cuts while avoiding the problems encountered by the Clean Development Mechanism (CDM). It proposes an approach to project evaluation based on measures of additionality, economic viability and contribution to sustainable development specific to this sector and re-evaluates 460 registered CDM projects in China that generate electricity using wind, natural gas or small hydro. It concludes that many small hydro schemes would be viable without the CDM subsidy. Although these projects have zero emissions from operations, offsets issued to projects that are viable without subsidy permit increased emissions in developed countries and lead to a net increase in global emissions. To provide some indication of the sustainable development benefits that CDM projects bring to their host countries, the paper includes estimates of the projects' benefit to public health due to the reduce